<a href="https://colab.research.google.com/github/roscoekerby/google-colab/blob/main/OpenAI_Zero_Shot_Model_CLIP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
!pip install transformers
#!pip install clip
!pip3 install clip-by-openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [97]:
import torch
import clip
from PIL import Image
import os 

local_download_path = os.path.expanduser('/content')
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)
text_array = ["a mouse", "a car", "an animal", "a human", "a dog", "a computer"]

print("For the input sequence: \"{}\"".format(text_array))

for query_file_name in os.listdir(local_download_path):
  if query_file_name.endswith("png") or query_file_name.endswith("jpg"): 
    image = preprocess(Image.open(query_file_name)).unsqueeze(0).to(device)
    text = clip.tokenize(text_array).to(device)

    with torch.no_grad():
        image_features = model.encode_image(image)
        text_features = model.encode_text(text)
        
        logits_per_image, logits_per_text = model(image, text)
        probs = logits_per_image.softmax(dim=-1).cpu().numpy()

        threshold_count = 0  # number of items that have greater scores than the threshold value
        threshold = 0.6 # theshold value
        label_index = 0

        for x in range(len(probs)):
          for y in range(len(probs[x])): 
            if probs[x][y] > threshold:
                threshold_count = threshold_count + 1
                label_index = y

                print("For the {} file, the Zero Shot Model predicted the following {} number of matches with their scores above the threshold of {}:".format(query_file_name, threshold_count, threshold))
                print("Label: \"{}\" with score {}".format(text_array[label_index], probs[x][y]))  # for some reason it cannot find the whale or mouse - probably no whales or mice in dataset

For the input sequence: "['a mouse', 'a car', 'an animal', 'a human', 'a dog', 'a computer']"
For the human-face.png file, the Zero Shot Model predicted the following 1 number of matches with their scores above the threshold of 0.6:
Label: "a human" with score 0.600774884223938
For the car.png file, the Zero Shot Model predicted the following 1 number of matches with their scores above the threshold of 0.6:
Label: "a car" with score 0.9485023617744446
For the mouse.jpg file, the Zero Shot Model predicted the following 1 number of matches with their scores above the threshold of 0.6:
Label: "a mouse" with score 0.9569043517112732
For the roscoe.jpg file, the Zero Shot Model predicted the following 1 number of matches with their scores above the threshold of 0.6:
Label: "a human" with score 0.7336633205413818
For the cat.png file, the Zero Shot Model predicted the following 1 number of matches with their scores above the threshold of 0.6:
Label: "an animal" with score 0.7027719020843506
